In [ ]:
import pandas as pd
import re
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## 네이버 부정 댓글 전처리

In [ ]:
! gdown 1IoNWZDzBZwQJIZZL6AnHUK2pVmkDSO4L # 네이버 부정댓글 크롤링한 데이터셋 불러오기

Downloading...
From: https://drive.google.com/uc?id=1IoNWZDzBZwQJIZZL6AnHUK2pVmkDSO4L
To: /content/neg_comment_naver_0620.csv
100% 5.99M/5.99M [00:00<00:00, 84.1MB/s]


In [ ]:
df = pd.read_csv('neg_comment_naver_0620.csv', index_col = 0)
df = df.T # 행,열 반전
df.drop_duplicates(['댓글'], inplace=True) # 중복제거
df

,별점,댓글
0,3,시계는 마음에 드는데... \n배터리가 없어서 바로 AS를 보냈습니다. 음... \...
1,3,택배받고 구매확정은 하였으나 상품을 확인해보니 작은바늘이 떨어져 있네요 \n선물한건...
2,3,넘 고급지고 이뻐요 ㅜ ㅠ 세일기간이라 좋은시계 저렴하게 득템했습니다ㅋㅋ
3,1,산지 하루만에 고장났어요...\n배터리가 다된거면 이해한다 싶어서 자주가던 세운스퀘...
4,1,색상도 디자인도 예쁜데 시계가 작동을 안하네요\n밧데리가 없나봐요ㅠ
...,...,...
55092,4,싼가격에잘산거같아요 이쁜ㅔ요
55093,4,면이랑 마소재고 얇아요 시원하겠어요~ 블루에 레드체크라 심심하지않아보이네요 면바지에...
55094,4,디자인 색상 모두 좋아요
55095,4,마음에 듭니다. 좋아요.제가 살이 쪄서 약간 작은듯하지만 신다보면 늘어나겠죠


In [ ]:
df.nunique() # 고유값 찾기

별점        5
댓글    51313
dtype: int64

In [ ]:
df.shape

(51313, 2)

In [ ]:
df.info() # 데이터프레임의 전반적인 정보 

<class 'pandas.core.frame.DataFrame'>
Index: 51313 entries, 0 to 55096
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   별점      51313 non-null  object
 1   댓글      51313 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [ ]:
df.dtypes

별점    object
댓글    object
dtype: object

In [ ]:
type(df['댓글'].iloc[0])

str

In [ ]:
## 정규표현식으로 전처리
def preprocess_sentence(sent):
  sent = re.sub(r"([?.!,¿])", r" \1", sent) # 구두점에 대해서 띄어쓰기
  sent = re.sub(r"[^ㄱ-ㅎ가-힣!.?]+", r" ", sent) # 한글과 구두점 제외하고 지우기
  sent = re.sub(r"\s+", " ", sent) # \s:space 를 표현하며 공백 문자를 의미, \s+ : \s가 한번 이상 반복 > ' '
  return sent

In [ ]:
df['댓글'] = df['댓글'].apply(str).apply(preprocess_sentence) # DataFrame에서 '댓글'만 정규표현식으로 전처리 적용

In [ ]:
print(df[df['별점'].str.contains('4', na=False)]) # 별점이 4점인 댓글만 추출

      별점                                                 댓글
6      4                             다소 무게감이 있지만 가성비 좋습니다 .
7      4                               크기는 살짝 큰것같고 착용하기 좋아요
9      4                                예쁘고 좋은듯해요 무게는 좀 있어요
10     4                       가성비 최고입니다 심플하면서 편하네요 디자인 굿 !
14     4  보통 신발을 을 신는데 리뷰를 보니 작다는 평도 있고 정사이즈 신으면 된다는 평도 ...
...   ..                                                ...
55092  4                                    싼가격에잘산거같아요 이쁜 요
55093  4  면이랑 마소재고 얇아요 시원하겠어요 블루에 레드체크라 심심하지않아보이네요 면바지에 ...
55094  4                                      디자인 색상 모두 좋아요
55095  4       마음에 듭니다 . 좋아요 .제가 살이 쪄서 약간 작은듯하지만 신다보면 늘어나겠죠
55096  4                           쿠션감이 폭신폭신해서 좋아요 신발도 예쁘네요

[37598 rows x 2 columns]


In [ ]:
# 별점이 4점인 댓글들 지우기 - 별점 3점 이하의 댓글들을 부정댓글로 고정
delet = df[df['별점'].str.contains('4', na=False)].index
df.drop(delet, inplace = True)
df

,별점,댓글
0,3,시계는 마음에 드는데 . . . 배터리가 없어서 바로 를 보냈습니다 . 음 . . ...
1,3,택배받고 구매확정은 하였으나 상품을 확인해보니 작은바늘이 떨어져 있네요 선물한건데 ...
2,3,넘 고급지고 이뻐요 세일기간이라 좋은시계 저렴하게 득템했습니다ㅋㅋ
3,1,산지 하루만에 고장났어요 . . . 배터리가 다된거면 이해한다 싶어서 자주가던 세운...
4,1,색상도 디자인도 예쁜데 시계가 작동을 안하네요 밧데리가 없나봐요
...,...,...
55077,3,사진과 똑같구요 박음질도 깔끔하고 가성비 좋은것같아요
55080,3,소재가 얇고 한철용입니다
55085,3,한달사용기천이 촉감이 좋아요
55086,3,너무 멋져요 감사합니다


In [ ]:
# 열 이름 바꾸기
df = df.rename(columns = {'별점' : 'star', '댓글' : 'comment'})
df = df[['comment', 'star']]
df

,comment,star
0,시계는 마음에 드는데 . . . 배터리가 없어서 바로 를 보냈습니다 . 음 . . ...,3
1,택배받고 구매확정은 하였으나 상품을 확인해보니 작은바늘이 떨어져 있네요 선물한건데 ...,3
2,넘 고급지고 이뻐요 세일기간이라 좋은시계 저렴하게 득템했습니다ㅋㅋ,3
3,산지 하루만에 고장났어요 . . . 배터리가 다된거면 이해한다 싶어서 자주가던 세운...,1
4,색상도 디자인도 예쁜데 시계가 작동을 안하네요 밧데리가 없나봐요,1
...,...,...
55077,사진과 똑같구요 박음질도 깔끔하고 가성비 좋은것같아요,3
55080,소재가 얇고 한철용입니다,3
55085,한달사용기천이 촉감이 좋아요,3
55086,너무 멋져요 감사합니다,3


In [ ]:
# 데이터프레임 저장
df.to_csv('/content/drive/MyDrive/Naver_neg_data_0621.csv',index=False)

In [ ]:
# !gdown 1u7r1lzpxzbFFwQFrc1X15IzaEhzTNVMN

Downloading...
From: https://drive.google.com/uc?id=1u7r1lzpxzbFFwQFrc1X15IzaEhzTNVMN
To: /content/Naver_neg_data_0621.csv
100% 1.36M/1.36M [00:00<00:00, 163MB/s]


In [ ]:
# test = pd.read_csv('Naver_neg_data_0621.csv')
# test

## 무신사 부정댓글 데이터와 네이버 부정댓글 데이터 합치기

In [ ]:
# 무신사 부정댓글 불러오기
!gdown 1fN0MKe-7_rqEC8biDDNwjBGUPcgA-XsZ

Downloading...
From: https://drive.google.com/uc?id=1fN0MKe-7_rqEC8biDDNwjBGUPcgA-XsZ
To: /content/preprocessing_ng_data.csv
100% 4.58M/4.58M [00:00<00:00, 48.5MB/s]


In [ ]:
df2 = pd.read_csv('preprocessing_ng_data.csv', index_col = 0)
df2

,comment,star,label
0,사이즈도 맞고 사진이랑 색도 같네요 옷도 이쁘고 마음에 들어요 마감이 살짝 아쉽긴 ...,4,1
1,너무 딱 맞아서 사이즈 바꿀려다가 입으면 늘어날 재질같아서 그냥 입기로했어요 도톰하...,4,1
2,기장이 많이 길진 않구요 . 엉덩이 가릴정도로 오버핏은 아니고 모자도 그렇게 크진않...,4,1
3,제가 올린 사진이 거지같아서 그렇지 실물이 더 예쁘긴 한데 색이 너무 바랜 원단이라...,4,1
4,그렇게 레쟈느낌이 강하지는 않고 핏도 무난 무난합니다 . 그런데 사진보다는 버튼이 ...,4,1
...,...,...,...
21131,만원 코트치고는 이쁘고 좋은 것 같네요 . 한 가지 아쉬운점은 내부주머니가 바느질...,4,1
21132,길이 두깨겜 전부 만족 ! 여름용으로도 만둘어쥬세요 또 살거니까요 !,4,1
21133,생각보다 너무 박시해서 당황했지만 옷이 넘기엽구 안이 기모라 넘넘 따뜻하네영 !,4,1
21134,부드러울거 생각했는데 그렇게 부드럽진않네요 근데 괜찮아요 색깔이 원하던 색깔이라 좋...,3,1


In [ ]:
# label열 제거
del df2['label']
df2

,comment,star
0,사이즈도 맞고 사진이랑 색도 같네요 옷도 이쁘고 마음에 들어요 마감이 살짝 아쉽긴 ...,4
1,너무 딱 맞아서 사이즈 바꿀려다가 입으면 늘어날 재질같아서 그냥 입기로했어요 도톰하...,4
2,기장이 많이 길진 않구요 . 엉덩이 가릴정도로 오버핏은 아니고 모자도 그렇게 크진않...,4
3,제가 올린 사진이 거지같아서 그렇지 실물이 더 예쁘긴 한데 색이 너무 바랜 원단이라...,4
4,그렇게 레쟈느낌이 강하지는 않고 핏도 무난 무난합니다 . 그런데 사진보다는 버튼이 ...,4
...,...,...
21131,만원 코트치고는 이쁘고 좋은 것 같네요 . 한 가지 아쉬운점은 내부주머니가 바느질...,4
21132,길이 두깨겜 전부 만족 ! 여름용으로도 만둘어쥬세요 또 살거니까요 !,4
21133,생각보다 너무 박시해서 당황했지만 옷이 넘기엽구 안이 기모라 넘넘 따뜻하네영 !,4
21134,부드러울거 생각했는데 그렇게 부드럽진않네요 근데 괜찮아요 색깔이 원하던 색깔이라 좋...,3


In [ ]:
df

In [ ]:
df2.to_csv('/content/drive/MyDrive/Musinsa_neg_data_0621.csv', index=False)

In [ ]:
# concat을 사용하여 세로방향으로 무신사 부정댓글df + 네이버 부정댓글df 
new_df = pd.concat([df, df2])
new_df

,comment,star
0,시계는 마음에 드는데 . . . 배터리가 없어서 바로 를 보냈습니다 . 음 . . ...,3
1,택배받고 구매확정은 하였으나 상품을 확인해보니 작은바늘이 떨어져 있네요 선물한건데 ...,3
2,넘 고급지고 이뻐요 세일기간이라 좋은시계 저렴하게 득템했습니다ㅋㅋ,3
3,산지 하루만에 고장났어요 . . . 배터리가 다된거면 이해한다 싶어서 자주가던 세운...,1
4,색상도 디자인도 예쁜데 시계가 작동을 안하네요 밧데리가 없나봐요,1
...,...,...
21131,만원 코트치고는 이쁘고 좋은 것 같네요 . 한 가지 아쉬운점은 내부주머니가 바느질...,4
21132,길이 두깨겜 전부 만족 ! 여름용으로도 만둘어쥬세요 또 살거니까요 !,4
21133,생각보다 너무 박시해서 당황했지만 옷이 넘기엽구 안이 기모라 넘넘 따뜻하네영 !,4
21134,부드러울거 생각했는데 그렇게 부드럽진않네요 근데 괜찮아요 색깔이 원하던 색깔이라 좋...,3


In [ ]:
new_df.to_csv('/content/drive/MyDrive/neg_pre_data_0621.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/neg_pre_data_0621.csv')
df

,comment,star
0,시계는 마음에 드는데 . . . 배터리가 없어서 바로 를 보냈습니다 . 음 . . ...,3
1,택배받고 구매확정은 하였으나 상품을 확인해보니 작은바늘이 떨어져 있네요 선물한건데 ...,3
2,넘 고급지고 이뻐요 세일기간이라 좋은시계 저렴하게 득템했습니다ㅋㅋ,3
3,산지 하루만에 고장났어요 . . . 배터리가 다된거면 이해한다 싶어서 자주가던 세운...,1
4,색상도 디자인도 예쁜데 시계가 작동을 안하네요 밧데리가 없나봐요,1
...,...,...
34846,만원 코트치고는 이쁘고 좋은 것 같네요 . 한 가지 아쉬운점은 내부주머니가 바느질...,4
34847,길이 두깨겜 전부 만족 ! 여름용으로도 만둘어쥬세요 또 살거니까요 !,4
34848,생각보다 너무 박시해서 당황했지만 옷이 넘기엽구 안이 기모라 넘넘 따뜻하네영 !,4
34849,부드러울거 생각했는데 그렇게 부드럽진않네요 근데 괜찮아요 색깔이 원하던 색깔이라 좋...,3
